In [28]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI

from langchain_core.prompts import ChatPromptTemplate

from langchain_core.runnables import RunnableLambda
import Helper
load_dotenv()

google_api_jey = os.getenv("GOOGLE_API_KEY")
oprn_api_key = os.getenv("OPEN_API_KEY")

google_llm = ChatGoogleGenerativeAI(temperature=0, api_key=google_api_jey, model="gemini-2.5-pro")

open_llm = ChatOpenAI(
    model="gpt-3.5-turbo", 
    temperature=0, 
    api_key=oprn_api_key,
    )



#LCEL - Single session

In [32]:

from langchain_core.messages import HumanMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant. Answer all the questions very shortly"),
    MessagesPlaceholder(variable_name="history"),
    ("human","{input}"),
])

chain = prompt_template |  open_llm

conversation=[]

def chat_with_bot(payload):
    print(payload)
    input = payload["input"]
    print(input)
    print(conversation)
    response = chain.invoke({"input": input, "history": conversation})
    print(response)
    

    conversation.append(HumanMessage(content=input))
    conversation.append(AIMessage(content=response.content))
    return response.content

res1 = chat_with_bot({"input" : "What is Langchain?"})
print(res1)
res2 = chat_with_bot({"input" : "Who is its founder?"})
print(res2) 

print(res1, "\n", res2)
   

{'input': 'What is Langchain?'}
What is Langchain?
[]
content='Langchain is a decentralized language learning platform that uses blockchain technology to connect language learners with native speakers for practice and tutoring.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 28, 'total_tokens': 52, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cg8Dl5WFe6htiTqMoDaEGzJ3X4pZb', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--8b8600d1-1b01-4ce6-bd3a-b5ab215b330f-0' usage_metadata={'input_tokens': 28, 'output_tokens': 24, 'total_tokens': 52, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0

In [ ]:
#LCEL Session Management with User level

In [35]:

from langchain_core.messages import HumanMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_classic.memory import ChatMessageHistory

prompt_template = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant. Answer all the questions very shortly"),
    MessagesPlaceholder(variable_name="history"),
    ("human","{input}"),
])

chain = prompt_template |  open_llm

conversation= {}

def chat_with_bot(payload):
    print(payload)
    session_id = payload["session_id"]
    input = payload["input"]
    print(input)

    
    if session_id not in conversation:
        conversation[session_id] = ChatMessageHistory()

    history = conversation[session_id]

    history.add_user_message(input)
    response = chain.invoke({"input": input, "history": history.messages})
    history.add_ai_message(response.content)
    print(response)

    return response.content

res1 = chat_with_bot({"input" : "What is Langchain?", "session_id": "user1"})
print(res1)
res2 = chat_with_bot({"input" : "Who is its founder?", "session_id": "user2"})
print(res2) 

print(res1, "\n", res2)

{'input': 'What is Langchain?', 'session_id': 'user1'}
What is Langchain?
content='Langchain is a decentralized autonomous organization (DAO) that aims to create a multilingual blockchain platform for cross-border communication and collaboration.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 37, 'total_tokens': 63, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cg8NsxGm3fZiOMMB5T4lGeFxxISfG', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--8729eacd-6be9-423b-9714-52bd56777cd0-0' usage_metadata={'input_tokens': 37, 'output_tokens': 26, 'total_tokens': 63, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_de

#Runnable with MessageHistory

In [ ]:

from langchain_core.messages import HumanMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_classic.memory import ChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser

prompt_template = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant. Answer all the questions very shortly"),
    MessagesPlaceholder(variable_name="history"),
    ("human","{input}"),
])

chain = prompt_template |  open_llm | StrOutputParser()

store ={}

def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

chain_with_message_history = RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key="input",
    history_messages_key="history")


res1 = chain_with_message_history.invoke(
    {"input" : "What is Google?"},
    config={"configurable": {"session_id": "user1"}}
)
print(res1)
res1 = chain_with_message_history.invoke(
    {"input" : "Where is it located?"},
    config={"configurable": {"session_id": "user1"}}
)
print(res1)

Langchain is a decentralized language learning platform that uses blockchain technology to connect language learners with native speakers for practice and tutoring.
Langchain is a digital platform, so it is accessible online from anywhere in the world.
